### Development of post processing

In [70]:
# Import python modules
import os
import sys
import pandas as pd
import gurobipy as gp
from time import time

# Determine the absolute path to the src directory (one level up from notebooks)
module_path = os.path.abspath(os.path.join("..", ""))
if module_path not in sys.path:
    sys.path.append(module_path)
print("module_path: ", module_path)

module_path:  c:\Users\tinus\OneDrive\Dokumenter\0 Master\code\master_project


In [71]:
runs_folder = os.path.join(
    os.path.dirname(os.path.abspath(os.getcwd())), "runs", "single_runs"
)
runs_folder

'c:\\Users\\tinus\\OneDrive\\Dokumenter\\0 Master\\code\\master_project\\runs\\single_runs'

In [72]:
import src.models as models

In [73]:
model_config_name = "small"
model_config = models.load_model_config(model_config_name)
print(model_config)

model_build_start_time = time()
model = models.get_model(model_config)
model_build_end_time = time()
print(f"Model build time: {model_build_end_time - model_build_start_time}")

model_run_start_time = time()
models.run_model(model)
model_run_end_time = time()
print(f"Model run time: {model_run_end_time - model_run_start_time}")

Path_or_name: small
Configuration loaded from c:\Users\tinus\OneDrive\Dokumenter\0 Master\code\master_project\configs\models\config_small.yaml
{'data_folder_name': 'elec_s_37_ES_PT', 'model_name': 'GTSEP_v0', 'model_id': 'test small', 'MIPGap': 0.01, 'VOLL': '6.35e3', 'CC': 100, 'CO2_price': 85, 'E_limit': inf, 'p_max_new_branch': 3000, 'p_min_new_branch': 100, 'expansion_factor': 2.0, 'MS': 0.1}
Set parameter MIPGap to value 0.001
Model build time: 31.099037408828735
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 332889 rows, 297856 columns and 840964 nonzeros
Model fingerprint: 0x2ba9ac17
Model has 70080 quadratic constraints
Variable types: 297847 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+05]
  QMatrix range    [8e-07, 1e

### Analyze run function from here 

In [94]:
def create_run_id(model_config: dict) -> str:
    from datetime import datetime

    now = datetime.now()
    formatted = now.strftime("%b%d:%a:h%H")
    model_name = model_config["model_name"]
    model_id = model_config["model_id"]
    now = datetime.now()
    formatted = now.strftime("%b%d_%a_h%H")
    run_id = f"{model_name}-{model_id}-{formatted}"
    return run_id


def save_model(model, model_config):
    run_id = create_run_id(model_config)
    single_run_folder = os.path.join(runs_folder, run_id)
    if not os.path.exists(single_run_folder):
        os.makedirs(single_run_folder)
    model_save_path = os.path.join(single_run_folder, "model" + ".mps")
    solution_save_path = os.path.join(single_run_folder, "solution" + ".sol")
    config_dump_save_path = os.path.join(single_run_folder, "config" + ".yaml")
    model.write(model_save_path)
    model.write(solution_save_path)
    import yaml

    model_config_dump = model_config.copy()
    model_config_dump["run_id"] = run_id

    with open(config_dump_save_path, "w") as file:
        yaml.dump(model_config_dump, file)

In [95]:
save_model(model, model_config)

In [96]:
base_names = {
    var.VarName.split("[")[0] if "[" in var.VarName else var.VarName
    for var in model.getVars()
}
print("Decision variable group names:", base_names)

Decision variable group names: {'soc', 'g_dis', 'p_b_max', 'x', 'z', 'sh', 'f', 'c', 'p_i_max', 'g', 'g_ch', 'y'}


In [ ]:
model_config

{'data_folder_name': 'elec_s_37_ES_PT',
 'model_name': 'GTSEP_v0',
 'model_id': 'test small',
 'MIPGap': 0.01,
 'VOLL': '6.35e3',
 'CC': 100,
 'CO2_price': 85,
 'E_limit': inf,
 'p_max_new_branch': 3000,
 'p_min_new_branch': 100,
 'expansion_factor': 2.0,
 'MS': 0.1}

In [7]:
import os
import gurobipy as gp

In [6]:
path = os.path.dirname(os.path.abspath(os.getcwd()))
folder_name = "GTSEP_v0-test small-Feb11_Tue_h09"
folder_path = os.path.join(path, "runs", "single_runs", folder_name)
model_info_path = os.path.join(folder_path, "model_info")

In [10]:
model = gp.read(os.path.join(model_info_path, "model.mps"))
model.read(os.path.join(model_info_path, "solution.sol"))

Read MPS format model from file c:\Users\tinus\OneDrive\Dokumenter\0 Master\code\master_project\runs\single_runs\GTSEP_v0-test small-Feb11_Tue_h09\model_info\model.mps
Reading time = 0.79 seconds
GTSEP_v0: 332889 rows, 297856 columns, 840964 nonzeros
Read solution from file c:\Users\tinus\OneDrive\Dokumenter\0 Master\code\master_project\runs\single_runs\GTSEP_v0-test small-Feb11_Tue_h09\model_info\solution.sol


In [11]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 332889 rows, 297856 columns and 840964 nonzeros
Model fingerprint: 0x91c09871
Model has 70080 quadratic constraints
Variable types: 297847 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+05]
  QMatrix range    [8e-07, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e-02, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-03, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Loaded user MIP start with objective 1.23737e+10

Presolve removed 201885 rows and 123425 columns
Presolve time: 0.52s
Presolved: 131025 rows, 174452 columns, 436463 nonzeros
Presolved model has 14 SOS constraint(s)
Variab